In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
cc = pd.read_csv("data_pack/cc.csv")
persons = pd.read_csv("data_pack/demographics.csv")
kplus = pd.read_csv("data_pack/kplus.csv")

In [3]:
outlier = list(cc["cc_txn_amt"][cc["cc_txn_amt"] > cc["cc_txn_amt"].quantile(.85)].index)
outlier2 = list(cc["cc_txn_amt"][cc["cc_txn_amt"] < cc["cc_txn_amt"].quantile(.15)].index)

In [4]:
persons.isnull().sum()

id         0
cc_no      0
gender     0
ocp_cd    26
age        0
dtype: int64

In [5]:
persons = persons.fillna(persons["ocp_cd"].mean())

In [6]:
for i in outlier2:
    outlier.append(i)

In [7]:
cc = cc.drop(outlier)

In [8]:
cc

,cc_no,pos_dt,cc_txn_amt
3,37069,2018-04-22,1600
5,37201,2018-04-13,400
7,37201,2018-01-15,600
8,37201,2018-06-13,1500
9,37201,2018-01-22,1300
...,...,...,...
999107,63420,2018-04-07,800
999108,63420,2018-01-23,800
999109,63420,2018-04-10,800
999111,63420,2018-02-11,1700


In [9]:
cc["pos_dt"] = pd.to_datetime(cc["pos_dt"],errors='coerce')

In [10]:
# cc["month"] = pd.DatetimeIndex(cc['pos_dt']).month

In [71]:
cc_2 = cc.groupby("cc_no").sum()

In [72]:
cc_2

,cc_txn_amt
cc_no,
2,8200
5,2000
7,9500
9,4000
11,12700
...,...
102403,28000
102406,4900
102407,23500


In [19]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

Using TensorFlow backend.


In [20]:
# define example
data = cc_2["pos_dt"]
data = array(data)
print(data)
# one hot encode
encoded = to_categorical(data)
print(encoded)
# invert encoding

[3 5 6 ... 3 4 5]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [21]:
df = pd.DataFrame(encoded , columns=["0","1","2","3","4","5","6"])

In [22]:
df = df.drop("0" , axis=1)

In [23]:
for i in range(len(df)):
    df.iloc[i] = df.iloc[i] * cc_2["cc_txn_amt"].iloc[i]
    

In [24]:
df["cc_no"] = cc_2["cc_no"]

In [25]:
df.groupby("cc_no").sum()

,1,2,3,4,5,6
cc_no,,,,,,
2,0.0,0.0,4600.0,0.0,1600.0,2000.0
5,2000.0,0.0,0.0,0.0,0.0,0.0
7,900.0,2300.0,2200.0,1200.0,2500.0,400.0
9,0.0,0.0,2000.0,0.0,2000.0,0.0
11,2500.0,2500.0,2500.0,3200.0,2000.0,0.0
...,...,...,...,...,...,...
102403,1500.0,1100.0,3000.0,8400.0,9800.0,4200.0
102406,0.0,500.0,1500.0,500.0,0.0,2400.0
102407,1900.0,2400.0,0.0,9400.0,9800.0,0.0


In [121]:
join1 = pd.merge(persons,df,on="cc_no", how="left")

In [122]:
join1 = join1.fillna(0)

In [123]:
join1 = join1.groupby(["id","cc_no","gender","ocp_cd","age"]).sum()

In [125]:
join1 = join1.join(cc_2, on="cc_no").fillna(0)

In [127]:
join1 = join1.reset_index()

In [130]:
join1 = join1.groupby(["id","gender","age","ocp_cd"]).sum()

***

In [80]:
kplus["sunday"] = pd.to_datetime(kplus["sunday"])

In [151]:
kplus.groupby("id").sum()

,kp_txn_count,kp_txn_amt
id,,
1,2,3500
2,104,210000
3,70,139700
4,315,6076100
6,10,13400
...,...,...
64995,88,960500
64996,20,41200
64997,76,201200


In [81]:
kplus_2 = kplus.groupby(["id", kplus.sunday.dt.month]).sum()

In [135]:
kplus_2

,id,sunday,kp_txn_count,kp_txn_amt
0,1,6,2,3500
1,2,1,13,7400
2,2,2,15,19600
3,2,3,14,23500
4,2,4,27,63400
...,...,...,...,...
272321,64999,2,20,192800
272322,64999,3,27,190700
272323,64999,4,23,70400
272324,64999,5,14,14400


In [170]:
# define example
data2 = array(kplus_2["sunday"])
# one hot encode
encoded2 = to_categorical(data2)


In [171]:
encoded2[0]

array([0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [172]:
df2 = pd.DataFrame(encoded2 , columns=["0","k_plus1","k_plus2","k_plus3","k_plus4","k_plus5","k_plus6"]).drop("0",axis=1)

In [173]:
df2

,k_plus1,k_plus2,k_plus3,k_plus4,k_plus5,k_plus6
0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
272321,0.0,1.0,0.0,0.0,0.0,0.0
272322,0.0,0.0,1.0,0.0,0.0,0.0
272323,0.0,0.0,0.0,1.0,0.0,0.0
272324,0.0,0.0,0.0,0.0,1.0,0.0


In [174]:
for i in range(len(df2)):
    df2.iloc[i] = df2.iloc[i] * kplus_2["kp_txn_amt"].iloc[i]
    

In [175]:
df2["id"] = kplus_2["id"]

***

In [176]:
df2 = df2.groupby("id").sum()

In [177]:
df2

,k_plus1,k_plus2,k_plus3,k_plus4,k_plus5,k_plus6
id,,,,,,
1,0.0,0.0,0.0,0.0,0.0,3500.0
2,7400.0,19600.0,23500.0,63400.0,24900.0,71200.0
3,15800.0,14700.0,20200.0,49900.0,16400.0,22700.0
4,1056700.0,859000.0,715300.0,1305500.0,729000.0,1410600.0
6,0.0,0.0,0.0,0.0,0.0,13400.0
...,...,...,...,...,...,...
64995,91700.0,48400.0,144600.0,84800.0,243300.0,347700.0
64996,0.0,3800.0,3300.0,3500.0,8800.0,21800.0
64997,48300.0,1000.0,40600.0,68800.0,35000.0,7500.0


In [178]:
persons_cc_kplus = join1.join(df2,on="id",how="left").fillna(0).drop("cc_no",axis=1)

In [180]:
persons_cc_kplus = persons_cc_kplus.join(kplus.groupby("id").sum(),on="id",how="left").drop("kp_txn_count",axis=1).fillna(0)

In [189]:
persons_cc_kplus = persons_cc_kplus.astype('int32')

***

In [182]:
income = pd.read_csv("C:/Users/WIN10/Desktop/work/TechJam/data_pack/train.csv")

In [190]:
Train_data = persons_cc_kplus.iloc[0:50000]

In [202]:
Train_data

,,,,1,2,3,4,5,6,cc_txn_amt,k_plus1,k_plus2,k_plus3,k_plus4,k_plus5,k_plus6,kp_txn_amt
id,gender,age,ocp_cd,,,,,,,,,,,,,,
1,2,5,9.0,0,0,0,0,0,0,0,0,0,0,0,0,3500,3500
2,2,4,3.0,6600,5700,8700,11700,1600,12200,46500,7400,19600,23500,63400,24900,71200,210000
3,2,3,1.0,0,0,0,0,0,0,0,15800,14700,20200,49900,16400,22700,139700
4,2,5,3.0,0,0,0,0,0,0,0,1056700,859000,715300,1305500,729000,1410600,6076100
5,2,4,9.0,2000,0,0,0,0,0,2000,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,2,4,3.0,1500,1600,0,1600,1600,1600,7900,6200,20500,17700,23100,27000,22000,116500
49997,1,5,9.0,0,0,0,0,0,3900,3900,124800,129700,135000,83700,146100,75300,694600
49998,2,2,9.0,0,4700,4700,1700,3400,600,15100,69400,15300,87900,88900,70700,55800,388000


In [191]:
Test_data = persons_cc_kplus.iloc[50000:]

***

In [192]:
import seaborn as sb
from sklearn.linear_model import LinearRegression

In [194]:
regressor = LinearRegression() 
regressor.fit(Train_data.iloc[:,1:],income["income"])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [195]:
print("Model intercept is", regressor.intercept_)
print("Model slope is", regressor.coef_)

Model intercept is 39175.60574143951
Model slope is [ 0.00135658 -0.03658744 -0.02251762 -0.31525045  0.44319695  0.54933587
  0.00093422 -0.00109648 -0.00360359  0.00551332 -0.00104614  0.00103852
  0.00173985]


In [196]:
predict_income = regressor.predict(Test_data.iloc[:,1:])

In [197]:
predict_income

array([41466.5940794 , 40238.20949667, 39876.08242872, ...,
       39660.57243109, 49149.77916175, 42327.98435756])

In [199]:
output = pd.read_csv("C:/Users/WIN10/Downloads/For problem download/data_pack/test.csv")

In [200]:
output["income"] = predict_income

In [201]:
output.to_csv(r"C:\Users\WIN10\Desktop\work\TechJam\splitMonth_output.csv",index=False)